In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [27]:
app = pd.read_csv("application_record.csv")
crecord = pd.read_csv("credit_record.csv")

In [28]:
app = app.drop_duplicates('ID', keep='last') 
# we identified that there are some duplicates in this dataset
# we will be deleting those duplicates and will keep the last entry of the ID if its repeated

In [29]:
# app.drop('OCCUPATION_TYPE', axis=1, inplace=True) 
#we identified earlier that occupation_type has many missing values
# we will drop this column

In [30]:
ot = pd.DataFrame(app.dtypes =='object').reset_index()
object_type = ot[ot[0] == True]['index']

#we are filtering the columns that have non numeric values to see if they are useful

In [31]:
num_type = pd.DataFrame(app.dtypes != 'object').reset_index().rename(columns =  {0:'yes/no'})
num_type = num_type[num_type['yes/no'] ==True]['index']
#HAVE CREATED SEPARATE LIST FOR NUMERIC TYPE INCASE IT WILL BE NEEDED IN FURTHER ANALYSIS
# IT IS NEEDED IN FURTHER ANALYSIS

In [32]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# for x in app:
#     if app[x].dtypes=='object':
#         app[x] = le.fit_transform(app[x])
# we have transformed all the non numeric data columns into data columns
# this method applies 0,1.. classification to different value types.

In [33]:
app

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,M,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,NaN,1.0
438553,6840222,F,N,N,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,1,0,0,0,Laborers,1.0
438554,6841878,F,N,N,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1,1,0,0,Sales staff,1.0
438555,6842765,F,N,Y,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,1,0,0,0,NaN,2.0


In [34]:
# Removing outliers
# FOR CNT_CHILDREN COLUMN
q_hi = app['CNT_CHILDREN'].quantile(0.999)
q_low = app['CNT_CHILDREN'].quantile(0.001)
app = app[(app['CNT_CHILDREN']>q_low) & (app['CNT_CHILDREN']<q_hi)]

In [35]:
# FOR AMT_INCOME_TOTAL COLUMN
q_hi = app['AMT_INCOME_TOTAL'].quantile(0.999)
q_low = app['AMT_INCOME_TOTAL'].quantile(0.001)
app= app[(app['AMT_INCOME_TOTAL']>q_low) & (app['AMT_INCOME_TOTAL']<q_hi)]

In [36]:
#FOR CNT_FAM_MEMBERS COLUMN
q_hi = app['CNT_FAM_MEMBERS'].quantile(0.999)
q_low = app['CNT_FAM_MEMBERS'].quantile(0.001)
app= app[(app['CNT_FAM_MEMBERS']>q_low) & (app['CNT_FAM_MEMBERS']<q_hi)]

In [37]:
crecord['Months from today'] = crecord['MONTHS_BALANCE']*-1
crecord = crecord.sort_values(['ID','Months from today'], ascending=True)
# we calculated months from today column to see how much old is the month
# we also sort the data according to ID and Months from today columns. 

In [38]:
crecord['STATUS'].replace({'C': 0, 'X' : 0}, inplace=True)
crecord['STATUS'] = crecord['STATUS'].astype('int')
crecord['STATUS'] = crecord['STATUS'].apply(lambda x:1 if x >= 2 else 0)
# replace the value C and X with 0 as it is the same type
# 1,2,3,4,5 are classified as 1 because they are the same type
# these will be our labels/prediction results for our model

In [39]:
crecordgb = crecord.groupby('ID').agg(max).reset_index()
#we are grouping the data in crecord by ID so that we can join it with app

In [40]:
df = app.join(crecordgb.set_index('ID'), on='ID', how='inner')
df.drop(['Months from today', 'MONTHS_BALANCE'], axis=1, inplace=True)
# no that this is joined, we will solve over sampling issue

In [22]:
def convert_dummy(df, feature,rank=0):
    pos = pd.get_dummies(df[feature], prefix=feature)
    mode = df[feature].value_counts().index[rank]
    biggest = feature + '_' + str(mode)
    pos.drop([biggest],axis=1,inplace=True)
    df.drop([feature],axis=1,inplace=True)
    df=df.join(pos)
    return df

In [23]:
df = convert_dummy(df,'NAME_INCOME_TYPE')
df = convert_dummy(df,'NAME_EDUCATION_TYPE')
df = convert_dummy(df,'NAME_FAMILY_STATUS')
df = convert_dummy(df,'NAME_HOUSING_TYPE')

In [24]:
X = df.iloc[:,1:-1] # X value contains all the variables except labels
y = df.iloc[:,-1] # these are the labels

In [41]:
df

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,STATUS
29,5008838,M,N,Y,1,405000.0,Commercial associate,Higher education,Married,House / apartment,-11842,-2016,1,0,0,0,Managers,3.0,0
30,5008839,M,N,Y,1,405000.0,Commercial associate,Higher education,Married,House / apartment,-11842,-2016,1,0,0,0,Managers,3.0,0
31,5008840,M,N,Y,1,405000.0,Commercial associate,Higher education,Married,House / apartment,-11842,-2016,1,0,0,0,Managers,3.0,0
32,5008841,M,N,Y,1,405000.0,Commercial associate,Higher education,Married,House / apartment,-11842,-2016,1,0,0,0,Managers,3.0,0
33,5008842,M,N,Y,1,405000.0,Commercial associate,Higher education,Married,House / apartment,-11842,-2016,1,0,0,0,Managers,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434779,5142964,M,N,N,1,180000.0,Working,Secondary / secondary special,Married,House / apartment,-10656,-926,1,1,1,0,Laborers,3.0,1
434780,5142972,M,N,N,1,180000.0,Working,Secondary / secondary special,Single / not married,House / apartment,-10656,-926,1,1,1,0,Laborers,3.0,1
434782,5143342,M,N,N,1,216000.0,Working,Secondary / secondary special,Married,Municipal apartment,-12946,-2418,1,0,0,1,Laborers,3.0,1
434791,5145846,F,N,Y,1,256500.0,State servant,Higher education,Married,House / apartment,-17594,-1315,1,0,1,0,Managers,3.0,1


In [43]:
df['STATUS'].value_counts()


0    9365
1     151
Name: STATUS, dtype: int64

In [45]:
INCOME_CHOICES = [['commercial', 'Commercial associate'],
                  ['working', 'Working'],
                  ['pensioner', 'Pensioner'],
                  ['state', 'State servant'],
                  ['student', 'Student']]

    
EDUCATION_CHOICES = [['degree', 'Academic degree'],
                     ['higher', 'Higher education'],
                     ['secondary', 'Secondary / secondary special'],
                    [ 'higher_incompl[ete', 'Incomplete higher'],
                    [ 'lower', 'Lower secondary']]

FAMILY_CHOICES =  [['civil', 'Civil marriage'],
                  ['married', 'Married'],
                  ['single', 'Single / not married'],
                  ['separated', 'Separated'],
                  ['widow', 'Widow']]
                
HOUSING_CHOICES = [['rented', 'Rented apartment'],
    ['house', 'House / apartment'],
    ['coop', 'Co-op apartment'],
    ['municipal', 'Municipal apartment'],
    ['office', 'Office apartment'],
    ['parents', 'With parents']]


In [46]:
def replace(word, array):
    for row in array:
        key = row[0]
        value = row[1]
        # print(word, "  ", value)
        if word == value:
            return key
        
    return array[-1][0]

In [50]:

## TO JSON FORMAT
final_array = []

for index, row in df.iterrows():
    default_object = {}
    default_object['model'] = "website.applicationrecord"
    default_object['pk'] = row['ID']
    default_object['fields'] = {}
    default_object['fields']['code_gender'] = row['CODE_GENDER']
    default_object['fields']['flag_own_car'] = True if row['FLAG_OWN_CAR'] == "Y" else False
    default_object['fields']['flag_own_realty'] = True if row['FLAG_OWN_REALTY'] == "Y" else False
    default_object['fields']['cnt_children'] = row['CNT_CHILDREN']
    default_object['fields']['amt_income_total'] = row['AMT_INCOME_TOTAL']
    default_object['fields']['days_birth'] = row['DAYS_BIRTH']
    default_object['fields']['days_employed'] = row['DAYS_EMPLOYED']
    default_object['fields']['flag_mobil'] = True if row['FLAG_MOBIL'] == "Y" else False
    default_object['fields']['flag_work_phone'] = True if row['FLAG_WORK_PHONE'] == "Y" else False
    default_object['fields']['flag_phone'] = True if row['FLAG_PHONE'] == "Y" else False
    default_object['fields']['flag_email'] = True if row['FLAG_EMAIL'] == "Y" else False
    
    if 'OCCUPATION_TYPE' in row:
        default_object['fields']['occupation_type'] = row['OCCUPATION_TYPE']
    else: 
        default_object['fields']['occupation_type'] = ''
    
    
    default_object['fields']['name_income_type'] = replace(row['NAME_INCOME_TYPE'],INCOME_CHOICES)
    default_object['fields']['name_education_type'] = replace(row['NAME_EDUCATION_TYPE'], EDUCATION_CHOICES)
    default_object['fields']['name_family_status'] = replace(row['NAME_FAMILY_STATUS'], FAMILY_CHOICES)
    default_object['fields']['name_housing_type'] = replace(row['NAME_HOUSING_TYPE'], HOUSING_CHOICES)
    default_object['fields']['cnt_fam_members'] = row['CNT_FAM_MEMBERS']
    default_object['fields']['status'] = row['STATUS']
    final_array.append(default_object)
    

import json
# Directly from dictionary
with open('../backend/website/fixtures/json_data.json', 'w') as outfile:
    json.dump(final_array, outfile)